In [ ]:
# 완성
# 긴급도는 임의의 숫자로 정해놓았을 때 목표하는 기준치의 최적의 가중치값 구하기(회귀)
# 가중치만 변수임(가중치만 학습함)
import numpy as np

total_time = 400  # 총 몇초 작업
process_time_per_package = 2  # 하나의 택배 처리 시간 (2초)
max_packages = total_time // process_time_per_package  # 총 x개의 택배를 처리 가능
rocket_goal = 130  # 로켓배송 목표 처리 개수
normal_goal = 70   # 일반배송 목표 처리 개수
SIZE = 200
rocket_priority = 20  # 로켓배송 긴급도
normal_priority = 0  # 일반배송 긴급도

# 손실 함수 정의 (로켓배송과 일반배송 목표 차이) MSE
def loss_function(rocket_processed, normal_processed):
    rocket_loss = ((rocket_processed - rocket_goal) / rocket_goal) ** 2
    normal_loss = ((normal_processed - normal_goal) / normal_goal) ** 2
    return rocket_loss + normal_loss

# 경사 하강법을 사용하여 alpha 값을 업데이트
def gradient_descent(alpha, learning_rate, max_iterations):
    for iteration in range(max_iterations):
        h = 1e-2  # 매우 작은 값
        rocket_processed, normal_processed = simulate(alpha)
        rocket_alpha_processed, normal_alpha_processed = simulate(alpha+h)

        loss = loss_function(rocket_processed, normal_processed)
        loss_alpha = loss_function(rocket_alpha_processed, normal_alpha_processed)

        # alpha에 대한 수치적 미분 (손실 함수의 변화율 계산)
        alpha_grad = numerical_derivative(loss_alpha, loss, h)

        # print("------------")
        # print(loss)
        # print(loss_alpha)
        # print(alpha_grad)

        # alpha 업데이트
        alpha = alpha - learning_rate * alpha_grad

        # 출력: 현재 iteration, alpha, 손실 값
        print(f"Iteration {iteration+1}: alpha = {alpha:.4f}, loss = {loss:.6f}")

        # 손실이 1e-6 이하로 작아지면 조기 종료
        if loss < 1e-6:
            break

    return alpha

# 시뮬레이션 함수: 주어진 alpha에 따라 로켓배송과 일반배송 처리
def simulate(alpha):
    rocket_processed_list = []
    normal_processed_list = []

    for k in range(300):
        # 각 택배의 대기 시간 (초기 대기 시간은 0)
        rocket_processed = 0
        normal_processed = 0
        rocket_wait_times = np.random.randint(1, SIZE+1, size=SIZE)  # 로켓배송 SIZE개 들어옴
        normal_wait_times = np.random.randint(1, SIZE+1, size=SIZE)  # 일반배송 SIZE개 들어옴

        for t in range(total_time):
            # 각 초마다 대기 시간이 1초씩 증가
            rocket_wait_times += 1
            normal_wait_times += 1

            # 우선순위 계산
            rocket_priority_scores = alpha * rocket_wait_times + (1 - alpha) * rocket_priority
            normal_priority_scores = alpha * normal_wait_times + (1 - alpha) * normal_priority

            # 최고 우선순위를 가진 택배 선택
            if np.max(rocket_priority_scores) >= np.max(normal_priority_scores):
                if rocket_processed < 200:
                    max_index = np.argmax(rocket_priority_scores)
                    rocket_processed += 1
                    rocket_wait_times[max_index] = -1000000  # 처리된 택배는 대기 시간에서 제외
            else:
                if normal_processed < 200:
                    max_index = np.argmax(normal_priority_scores)
                    normal_processed += 1
                    normal_wait_times[max_index] = -1000000

            # 2초마다 하나의 택배를 처리
            if t % process_time_per_package == 0 and (rocket_processed + normal_processed) >= max_packages:
                break

        rocket_processed_list.append(rocket_processed)
        normal_processed_list.append(normal_processed)

    return int(sum(rocket_processed_list)/len(rocket_processed_list)), int(sum(normal_processed_list)/len(normal_processed_list))

# 수치적 미분 계산 (손실 함수의 변화율)
def numerical_derivative(alphaPlus_loss, alpha_loss, h):
    return (alphaPlus_loss - alpha_loss) / h

# 초기값 설정
initial_alpha = 0.5  # alpha 초기값
learning_rate = 0.01  # 학습률
max_iterations = 100  # 최대 반복 횟수

# 경사 하강법 실행
optimal_alpha = gradient_descent(initial_alpha, learning_rate, max_iterations)
print(f"최적의 alpha 값: {optimal_alpha:.4f}")

# for i in range(1, 10):
#   print(simulate(i*0.1))
# print(simulate(0.2541))


Iteration 1: alpha = 0.5000, loss = 0.105301
Iteration 2: alpha = 0.4897, loss = 0.095034
Iteration 3: alpha = 0.5000, loss = 0.105301
Iteration 4: alpha = 0.4897, loss = 0.095034
Iteration 5: alpha = 0.4897, loss = 0.105301
Iteration 6: alpha = 0.4795, loss = 0.095034
Iteration 7: alpha = 0.4692, loss = 0.095034
Iteration 8: alpha = 0.4692, loss = 0.095034
Iteration 9: alpha = 0.4692, loss = 0.095034
Iteration 10: alpha = 0.4595, loss = 0.085294
Iteration 11: alpha = 0.4497, loss = 0.085294
Iteration 12: alpha = 0.4497, loss = 0.085294
Iteration 13: alpha = 0.4497, loss = 0.085294
Iteration 14: alpha = 0.4497, loss = 0.085294
Iteration 15: alpha = 0.4400, loss = 0.085294
Iteration 16: alpha = 0.4308, loss = 0.076080
Iteration 17: alpha = 0.4308, loss = 0.076080
Iteration 18: alpha = 0.4308, loss = 0.076080
Iteration 19: alpha = 0.4308, loss = 0.076080
Iteration 20: alpha = 0.4216, loss = 0.076080
Iteration 21: alpha = 0.4129, loss = 0.067393
Iteration 22: alpha = 0.4047, loss = 0.0592

In [ ]:
# 긴급도 변수, 목표하는 기준치의 최적의 가중치값 구하기(회귀)
# 가중치, 긴급도 모두 학습
import numpy as np

total_time = 400  # 총 몇초 작업
process_time_per_package = 2  # 하나의 택배 처리 시간 (2초)
max_packages = total_time // process_time_per_package  # 총 x개의 택배를 처리 가능
rocket_goal = 130  # 로켓배송 목표 처리 개수
normal_goal = 70   # 일반배송 목표 처리 개수
SIZE = 200
# rocket_priority = 20  # 로켓배송 긴급도
normal_priority = 0  # 일반배송 긴급도

# 손실 함수 정의 (로켓배송과 일반배송 목표 차이) MSE
def loss_function(rocket_processed, normal_processed):
    rocket_loss = ((rocket_processed - rocket_goal) / rocket_goal) ** 2
    normal_loss = ((normal_processed - normal_goal) / normal_goal) ** 2
    return rocket_loss + normal_loss

# 경사 하강법을 사용하여 alpha 값을 업데이트
def gradient_descent(alpha,rocket_priority, learning_rate, max_iterations):
    for iteration in range(max_iterations):
        h = 1e-2  # 매우 작은 값
        rocket_processed, normal_processed = simulate(alpha-h, rocket_priority)
        rocket_alpha_processed, normal_alpha_processed = simulate(alpha+h, rocket_priority)

        rocket_priority_processed_minus, normal_priority_processed_minus = simulate(alpha, rocket_priority-h)
        rocket_priority_processed, normal_priority_processed = simulate(alpha, rocket_priority+h)

        loss_minus = loss_function(rocket_processed, normal_processed)
        loss_alpha = loss_function(rocket_alpha_processed, normal_alpha_processed)

        loss_priority_minus = loss_function(rocket_priority_processed_minus, normal_priority_processed_minus)
        loss_priority = loss_function(rocket_priority_processed, normal_priority_processed)

        # alpha에 대한 수치적 미분 (손실 함수의 변화율 계산)
        alpha_grad = numerical_derivative(loss_alpha, loss_minus, h)
        priority_grad = numerical_derivative(loss_priority, loss_priority_minus, h)

        # print("------------")
        # print(loss)
        # print(loss_alpha)
        # print(alpha_grad)

        # alpha 업데이트
        alpha = alpha - learning_rate * alpha_grad
        rocket_priority = rocket_priority - learning_rate * 10 * priority_grad

        loss_rocket_processed, loss_normal_processed = simulate(alpha, rocket_priority)
        loss = loss_function(loss_rocket_processed, loss_normal_processed)

        # 출력: 현재 iteration, alpha, 손실 값
        print(f"Iteration {iteration+1}: alpha = {alpha:.4f}, rocket_priority = {rocket_priority:.4f}, loss = {loss:.6f}")

        # 손실이 1e-6 이하로 작아지면 조기 종료
        if loss < 1e-6:
            break

    return alpha, rocket_priority

# 시뮬레이션 함수: 주어진 alpha에 따라 로켓배송과 일반배송 처리
def simulate(alpha, rocket_priority):
    rocket_processed_list = []
    normal_processed_list = []

    for k in range(300):
        # 각 택배의 대기 시간 (초기 대기 시간은 0)
        rocket_processed = 0
        normal_processed = 0
        rocket_wait_times = np.random.randint(1, SIZE+1, size=SIZE)  # 로켓배송 SIZE개 들어옴
        normal_wait_times = np.random.randint(1, SIZE+1, size=SIZE)  # 일반배송 SIZE개 들어옴

        for t in range(total_time):
            # 각 초마다 대기 시간이 1초씩 증가
            rocket_wait_times += 1
            normal_wait_times += 1

            # 우선순위 계산
            rocket_priority_scores = alpha * rocket_wait_times + (1 - alpha) * rocket_priority
            normal_priority_scores = alpha * normal_wait_times + (1 - alpha) * normal_priority

            # 최고 우선순위를 가진 택배 선택
            if np.max(rocket_priority_scores) >= np.max(normal_priority_scores):
                if rocket_processed < 200:
                    max_index = np.argmax(rocket_priority_scores)
                    rocket_processed += 1
                    rocket_wait_times[max_index] = -1000000  # 처리된 택배는 대기 시간에서 제외
            else:
                if normal_processed < 200:
                    max_index = np.argmax(normal_priority_scores)
                    normal_processed += 1
                    normal_wait_times[max_index] = -1000000

            # 2초마다 하나의 택배를 처리
            if t % process_time_per_package == 0 and (rocket_processed + normal_processed) >= max_packages:
                break

        rocket_processed_list.append(rocket_processed)
        normal_processed_list.append(normal_processed)

    return int(sum(rocket_processed_list)/len(rocket_processed_list)), int(sum(normal_processed_list)/len(normal_processed_list))

# 수치적 미분 계산 (손실 함수의 변화율)
def numerical_derivative(alphaPlus_loss, alphaMinus_loss, h):
    return (alphaPlus_loss - alphaMinus_loss) / (2*h)

# 초기값 설정
initial_alpha = np.random.uniform(0, 1)  # alpha 초기값
learning_rate = 0.01  # 학습률
max_iterations = 100  # 최대 반복 횟수
initial_rocket_priority = np.random.randint(5, 100) # rocket_priority 초기값

# 경사 하강법 실행
optimal_alpha, optimal_rocket_priority = gradient_descent(initial_alpha,initial_rocket_priority, learning_rate, max_iterations)
print(f"최적의 alpha 값: {optimal_alpha:.4f}, 최적의 rocket_priority 값: {optimal_rocket_priority:.4f}")

# for i in range(1, 10):
#   print(simulate(i*0.1))
# print(simulate(0.2541))


Iteration 1: alpha = 0.8553, rocket_priority = 95.0592, loss = 0.116095
Iteration 2: alpha = 0.8494, rocket_priority = 95.0592, loss = 0.127415
Iteration 3: alpha = 0.8378, rocket_priority = 95.1158, loss = 0.116095
Iteration 4: alpha = 0.8280, rocket_priority = 95.1698, loss = 0.105301
Iteration 5: alpha = 0.8229, rocket_priority = 95.1698, loss = 0.105301
Iteration 6: alpha = 0.8178, rocket_priority = 95.1698, loss = 0.105301
Iteration 7: alpha = 0.8126, rocket_priority = 95.1698, loss = 0.095034
Iteration 8: alpha = 0.8026, rocket_priority = 95.1698, loss = 0.085294
Iteration 9: alpha = 0.7977, rocket_priority = 95.1698, loss = 0.076080
Iteration 10: alpha = 0.7931, rocket_priority = 95.2159, loss = 0.085294
Iteration 11: alpha = 0.7837, rocket_priority = 95.2159, loss = 0.067393
Iteration 12: alpha = 0.7793, rocket_priority = 95.2159, loss = 0.076080
Iteration 13: alpha = 0.7709, rocket_priority = 95.2159, loss = 0.067393
Iteration 14: alpha = 0.7665, rocket_priority = 95.2159, los

In [ ]:
print(simulate(0.6187, 95.1790))

(130, 70)
